El archivo movies_dataset.csv contiene información detallada sobre películas, incluyendo datos sobre su reparto (cast), equipo de producción (crew), identificadores únicos (id) y posiblemente otras características relevantes para su análisis.

Cada película está representada por una fila, y las columnas pueden incluir estructuras anidadas en formato de texto JSON, lo que requiere procesamiento para extraer la información correctamente.

En este notebook se realiza una limpieza y transformación del dataset de películas (movies_dataset.csv). A continuación, se detallan los pasos principales:

    Carga de datos: Se lee el archivo CSV original.

    Desanidación de datos: Se extraen los nombres de belongs_to_collection, production_companies y production_countries, que están almacenados como listas de diccionarios.

    Manejo de valores nulos:

        Se reemplazan los valores nulos en revenue y budget con 0.

        Se eliminan las filas sin fecha de estreno (release_date).

    Formateo de fechas:

        Se convierte release_date al formato AAAA-mm-dd.

        Se extrae el release_year en una nueva columna.

    Cálculo del retorno de inversión: Se crea la columna return, calculando revenue / budget, asignando 0 cuando budget es 0 para evitar errores.

    Eliminación de columnas innecesarias: Se eliminan video, imdb_id, adult, original_title, poster_path y homepage para optimizar el dataset.

    Reducción del dataset: Se elimina aleatoriamente el 40% de los registros para reducir el tamaño y mantener consistencia con otros datasets procesados.

    Exportación: El dataset limpio se guarda en formato Parquet en la carpeta dataset_limpio, mejorando el rendimiento y el almacenamiento.


In [2]:
import pandas as pd
import os
import json
import ast


In [3]:

# Ruta del archivo en tu sistema local
file_path = r"C:\Users\E B M\Documents\proyecto_mp\dataset_org\movies_dataset.csv"

# Cargar el archivo
df = pd.read_csv(file_path, low_memory=False)



In [4]:
# Mostrar información general del DataFrame
print("Información general del DataFrame:")
print(df.info())

Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date       

In [5]:
# Mostrar las primeras filas del DataFrame
print("\nPrimeras filas del DataFrame:")
print(df.head())


Primeras filas del DataFrame:
   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                  

In [6]:
# Descripción estadística de las columnas numéricas
print("\nDescripción estadística:")
print(df.describe())


Descripción estadística:
            revenue       runtime  vote_average    vote_count
count  4.546000e+04  45203.000000  45460.000000  45460.000000
mean   1.120935e+07     94.128199      5.618207    109.897338
std    6.433225e+07     38.407810      1.924216    491.310374
min    0.000000e+00      0.000000      0.000000      0.000000
25%    0.000000e+00     85.000000      5.000000      3.000000
50%    0.000000e+00     95.000000      6.000000     10.000000
75%    0.000000e+00    107.000000      6.800000     34.000000
max    2.787965e+09   1256.000000     10.000000  14075.000000


In [7]:
# Mostrar la cantidad de valores nulos por columna
print("\nValores nulos por columna:")
print(df.isnull().sum())

# Mostrar el tipo de datos de cada columna
print("\nTipos de datos de cada columna:")
print(df.dtypes)


Valores nulos por columna:
adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

Tipos de datos de cada columna:
adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id               

In [8]:
print(df['title'].head())  # Muestra las primeras 5 películas


0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object


In [9]:
# 1. Desanidar columnas con diccionarios o listas
def extract_name(value):
    if pd.isna(value) or value == "[]" or value == "{}":
        return None
    try:
        data = ast.literal_eval(value)
        if isinstance(data, dict):
            return data.get('name', None)
        elif isinstance(data, list):
            return ', '.join([item['name'] for item in data if 'name' in item])
    except Exception as e:
        return None
    return None

df['belongs_to_collection'] = df['belongs_to_collection'].apply(extract_name)
df['production_companies'] = df['production_companies'].apply(extract_name)
df['production_countries'] = df['production_countries'].apply(extract_name)
df['genres'] = df['genres'].apply(extract_name)
df['spoken_languages'] = df['spoken_languages'].apply(extract_name)



In [10]:

# 2. Rellenar valores nulos en revenue y budget con 0
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)


In [11]:

# 3. Eliminar valores nulos en release_date
df = df.dropna(subset=['release_date'])


In [12]:

# 4. Formatear fechas y extraer el anio de estreno
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year


In [13]:
# Convertir 'revenue' y 'budget' a valores numéricos, reemplazando errores con 0
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce').fillna(0)
df['budget'] = pd.to_numeric(df['budget'], errors='coerce').fillna(0)

# Crear la columna return
df['return'] = df.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] > 0 else 0, axis=1)


In [14]:
# Verificar qué columnas existen antes de eliminarlas
columns_to_drop = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
existing_columns = [col for col in columns_to_drop if col in df.columns]

# Eliminar solo las columnas que existen
df.drop(columns=existing_columns, inplace=True)


In [15]:
# Seleccionar columnas clave para mostrar información
important_columns = ['title', 'release_year', 'genres', 'budget', 'revenue', 'return', 'popularity', 'vote_average', 'vote_count']

# Mostrar las primeras filas de las columnas importantes
print(df[important_columns].head())

# Mostrar información general de las columnas importantes
print(df[important_columns].info())

# Descripción estadística de las columnas numéricas importantes
print(df[important_columns].describe())


                         title  release_year                      genres  \
0                    Toy Story        1995.0   Animation, Comedy, Family   
1                      Jumanji        1995.0  Adventure, Fantasy, Family   
2             Grumpier Old Men        1995.0             Romance, Comedy   
3            Waiting to Exhale        1995.0      Comedy, Drama, Romance   
4  Father of the Bride Part II        1995.0                      Comedy   

       budget      revenue     return popularity  vote_average  vote_count  
0  30000000.0  373554033.0  12.451801  21.946943           7.7      5415.0  
1  65000000.0  262797249.0   4.043035  17.015539           6.9      2413.0  
2         0.0          0.0   0.000000    11.7129           6.5        92.0  
3  16000000.0   81452156.0   5.090760   3.859495           6.1        34.0  
4         0.0   76578911.0   0.000000   8.387519           5.7       173.0  
<class 'pandas.core.frame.DataFrame'>
Index: 45379 entries, 0 to 45465
Data colum

In [45]:

# Guardar el archivo limpio
df.to_parquet('C:/Users/E B M/Documents/proyecto_mp/dataset_limpio/movies_dataset_cleaned.parquet', index=False)
